In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os
import sys

import keras
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.initializers import Constant
from keras.layers import (Activation, Conv2D, Conv3D, Dense, Dropout, Flatten,
                          Input, Lambda, MaxPooling2D)
from keras.models import Model
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from PIL import Image
from scipy import ndimage
from skimage.restoration import (denoise_bilateral, denoise_tv_chambolle,
                                 denoise_wavelet, estimate_sigma)
from sklearn.model_selection import StratifiedShuffleSplit
from vis.regularizers import LPNorm, TotalVariation
from vis.utils import utils
from vis.visualization import (visualize_activation,
                               visualize_activation_with_losses)
from tensorflow.contrib import graph_editor as ge
from cleverhans.attacks import (FastGradientMethod, MadryEtAl,
                                ProjectedGradientDescent, SparseL1Descent)
from cleverhans.model import Model as CHModel

sys.path.append('..')

from attribution.InfluenceInvariants import InfluenceInvariants
from attribution.ActivationInvariants import ActivationInvariants
from attribution.invariant_utils import (probits_from_invariants,
                                         smooth_logits_from_invariants,
                                         smooth_logit_tensor_from_invariants,
                                         smooth_logit_tensor_from_invariant,
                                         smooth_probits_from_invariants,
                                         smooth_probit_tensor_from_invariants,
                                         tally_total_stats)

logging.captureWarnings(True)
logging.getLogger('tensorflow').setLevel(logging.CRITICAL)
logging.getLogger('cleverhans').setLevel(logging.CRITICAL)

Using TensorFlow backend.


In [3]:
K.set_image_data_format('channels_last')

In [4]:
from sklearn.datasets import fetch_olivetti_faces, fetch_lfw_people

# Use only classes that have at least 100 images
# There are five such classes in LFW
lfw_slice = (slice(68, 196, None), slice(61, 190, None))
faces_data = fetch_lfw_people(min_faces_per_person=100, color=True, slice_=lfw_slice)
images = faces_data.images
n_classes = faces_data.target.max()+1
x, y = faces_data.data, keras.utils.to_categorical(faces_data.target, n_classes)
images /= 255.0

# Use 3/4 for training, the rest for testing
N_tr = int(len(x)*0.75)
N_te = len(x) - N_tr
x_tr, y_tr = x[:N_tr], y[:N_tr]
x_te, y_te = x[N_tr:], y[N_tr:]
im_tr, im_te = images[:N_tr], images[N_tr:]

In [5]:
inp = keras.layers.Input(shape=im_tr[0].shape, name='features')
out = keras.layers.Conv2D(128, (3,3), activation='relu')(inp)
out = keras.layers.MaxPooling2D(pool_size=(2,2))(out)
out = keras.layers.Conv2D(64, (3,3), activation='relu')(out)
out = keras.layers.MaxPooling2D(pool_size=(2,2))(out)
out = keras.layers.Conv2D(32, (3,3), activation='relu')(out)
out = keras.layers.MaxPooling2D(pool_size=(2,2))(out)
out = keras.layers.Conv2D(16, (3,3), activation='relu')(out)
out = keras.layers.MaxPooling2D(pool_size=(2,2))(out)
out = keras.layers.Flatten()(out)
out = keras.layers.Dense(16, activation='relu')(out)
out = keras.layers.Dense(y[0].shape[0], name='logits')(out)
out = keras.layers.Activation('softmax', name='softmax')(out)
model = keras.Model(inp, out)
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model.summary()
model.load_weights('weights/lfw-small-tf.h5')
print('accuracy:')
print('train={:.2}'.format(model.evaluate(im_tr, y_tr, verbose=False)[1]))
print('test={:.2}'.format(model.evaluate(im_te, y_te, verbose=False)[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
__________

The method below, `get_random_invariants`, does the following:
1. Samples $n$ uniform-random points from [0,1] into $x$
2. For each class label $l$, runs projected gradient descent (`MadryEtAl` from `cleverhans`) with both $L_\infty$ and $L_2$ norm over $x$ to create approximately $2n$ points with label $l$, stored in $x_l$
3. Concatenates all the labeled random points $x_1, \ldots, x_l$ into $x'$
4. For each layer $t$ specified in `layers`, generate a set of invariants using $x'$

Importantly, for influence invariants we exclude activation information (i.e., `multiply_attributions=False` in the invariant constructor). The reason for this will become clear later.

In [6]:
class KerasModel(CHModel):
    def __init__(self, model, **kwargs):
        del kwargs
        CHModel.__init__(self, 'model_b', model.output_shape[1], locals())

        self.model = model
        self.fprop(tf.placeholder(tf.float32, (128,)+model.input_shape[1:]))

    def fprop(self, x, **kwargs):
        del kwargs
        if isinstance(x, np.ndarray):
            x = K.variable(x)
        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            return dict([(layer.name, keras.Model(self.model.input, layer.output)(x)) for layer in self.model.layers])

def get_random_invariants(model, layers, n, x_shape, n_classes, nbiter=50, eps_linf=0.1, eps_l2=5., agg_fn=None, seed=0):
    np.random.seed(seed)
    rand_data = []
    cls_size = n
    rand_x = np.random.uniform(size=(cls_size,) + x_shape)
    nbiter = 50
    for l in range(n_classes):
        model_ch = KerasModel(model)
        sess = K.get_session()
        pgd_params_l2 = {'eps': eps_l2,
                         'y_target': to_categorical(np.zeros((1,)) + l, n_classes),
                         'eps_iter': eps_l2 / nbiter,
                         'nb_iter': nbiter,
                         'ord': 2,
                         'clip_min': 0.,
                         'clip_max': 1.}
        pgd_params_linf = {'eps': eps_linf,
                           'y_target': to_categorical(np.zeros((1,)) + l, n_classes),
                           'eps_iter': eps_linf / nbiter,
                           'nb_iter': nbiter,
                           'ord': np.inf,
                           'clip_min': 0.,
                           'clip_max': 1.}
        pgd = MadryEtAl(model_ch, sess=sess)
        cur_data = []
        cur_data.append(pgd.generate_np(rand_x, **pgd_params_l2))
        cur_data.append(pgd.generate_np(rand_x, **pgd_params_linf))
        rand_data.append(np.concatenate(cur_data, axis=0))
        print('generated class', l)

    rand_data = np.concatenate(rand_data)

    log_model = Model(model.inputs, model.layers[-2].output)
    log_model.layers[-1].activation = keras.activations.softplus
    log_model = utils.apply_modifications(log_model)

    gens = [InfluenceInvariants(log_model, layer=target, agg_fn=agg_fn,
                                multiply_activation=False).compile() for target in layers]
    invs_by_layer = [gen.get_invariants(rand_data, batch_size = 1) for gen in gens]

    return invs_by_layer, gens

# Influence invariants

We'll start with `agg_fn=None`, so the invariants refer to specific locations on the feature maps. We'll test all convolutional layers (1-4).

In [7]:
layers = [7,8]

In [99]:
invs_by_layer, gens = get_random_invariants(model, 
                                            layers, 
                                            256, 
                                            im_tr[0].shape, 
                                            5, 
                                            agg_fn=None)

generated class 0
generated class 1
generated class 2
generated class 3
generated class 4


TypeError: super(type, obj): obj must be an instance or subtype of type

In [12]:
len(invs_by_layer[0]), len(invs_by_layer[1])

(16, 13)

In [13]:
print(invs_by_layer[0][0])
print(invs_by_layer[1][0])

(conv2d_4[133] > 0.0 &
 conv2d_4[43] > 0.0 &
 conv2d_4[1] > 0.0)
	--> Q = 4
support=0.0148, precision=1.0
(max_pooling2d_4[42] > 0.0 &
 max_pooling2d_4[43] > 0.0 &
 max_pooling2d_4[40] > 0.0 &
 max_pooling2d_4[18] > 0.0)
	--> Q = 0
support=0.00193, precision=1.0


In [14]:
x_tr, x_te = im_tr, im_te
inv_models = [probits_from_invariants(invs) for invs in invs_by_layer]
for l in range(len(layers)):
    invs = invs_by_layer[l]
    inv_model = inv_models[l]
    print('-' * 10, 'training set results, layer', layers[l])
    n_per, sup, prec = tally_total_stats(
        invs, model, x_tr, batch_size=1)
    print('# invariants per class:', n_per)
    print('support by class:', sup)
    print('precision by class:', prec)
    print('overall prediction accuracy: {:.3}'.format(
        (inv_model(x_tr).argmax(axis=1) == y_tr.argmax(axis=1)).mean()))

    print('-' * 10, 'test set results, layer', layers[l])
    n_per, sup, prec = tally_total_stats(
        invs, model, x_te, batch_size=1)
    print('# invariants per class:', n_per)
    print('support by class:', sup)
    print('precision by class:', prec)
    print('overall prediction accuracy: {:.3}'.format(
        (inv_model(x_te).argmax(axis=1) == y_te.argmax(axis=1)).mean()))

---------- training set results, layer 7
# invariants per class: {0: 4, 1: 5, 2: 4, 3: 1, 4: 2}
support by class: {0: 0.9836956521739131, 1: 0.9148936170212766, 2: 0.9974811083123426, 3: 0.8888888888888888, 4: 0.8585858585858586}
precision by class: {0: 0.8916256157635468, 1: 1.0, 2: 0.9974811083123426, 3: 0.972972972972973, 4: 0.8947368421052632}
overall prediction accuracy: 0.959
---------- test set results, layer 7
# invariants per class: {0: 4, 1: 5, 2: 4, 3: 1, 4: 2}
support by class: {0: 1.0, 1: 0.9142857142857143, 2: 1.0, 3: 0.9705882352941176, 4: 0.8787878787878788}
precision by class: {0: 0.8727272727272727, 1: 1.0, 2: 1.0, 3: 1.0, 4: 0.9666666666666667}
overall prediction accuracy: 0.804
---------- training set results, layer 8
# invariants per class: {0: 3, 1: 3, 2: 4, 3: 2, 4: 1}
support by class: {0: 1.0, 1: 0.9893617021276596, 2: 0.9924433249370277, 3: 0.7530864197530864, 4: 1.0}
precision by class: {0: 0.9945945945945946, 1: 0.9893617021276596, 2: 1.0, 3: 1.0, 4: 0.83193

#### Summary: random influents invariants at layers 7, 8 achieve 95-97% training accuracy, and 80% test accuracy. The original model achieved 100% training accuracy and 81% test accuracy.

Now we'll generate influence invariants with `agg_fn=K.sum`

In [16]:
invs_by_layer, gens = get_random_invariants(model, 
                                            layers, 
                                            256, 
                                            im_tr[0].shape, 
                                            5, 
                                            agg_fn=K.sum)

generated class 0
generated class 1
generated class 2
generated class 3
generated class 4


In [17]:
x_tr, x_te = im_tr, im_te
inv_models = [probits_from_invariants(invs) for invs in invs_by_layer]
for l in range(len(layers)):
    invs = invs_by_layer[l]
    inv_model = inv_models[l]
    print('-' * 10, 'training set results, layer', layers[l])
    n_per, sup, prec = tally_total_stats(
        invs, model, x_tr, batch_size=1)
    print('# invariants per class:', n_per)
    print('support by class:', sup)
    print('precision by class:', prec)
    print('overall prediction accuracy: {:.3}'.format(
        (inv_model(x_tr).argmax(axis=1) == y_tr.argmax(axis=1)).mean()))

    print('-' * 10, 'test set results, layer', layers[l])
    n_per, sup, prec = tally_total_stats(
        invs, model, x_te, batch_size=1)
    print('# invariants per class:', n_per)
    print('support by class:', sup)
    print('precision by class:', prec)
    print('overall prediction accuracy: {:.3}'.format(
        (inv_model(x_te).argmax(axis=1) == y_te.argmax(axis=1)).mean()))

---------- training set results, layer 7
# invariants per class: {0: 3, 1: 3, 2: 4, 3: 6, 4: 3}
support by class: {0: 0.717391304347826, 1: 0.7021276595744681, 2: 0.7531486146095718, 3: 0.9382716049382716, 4: 0.9393939393939394}
precision by class: {0: 0.9850746268656716, 1: 0.9705882352941176, 2: 0.9835526315789473, 3: 0.39790575916230364, 4: 1.0}
overall prediction accuracy: 0.836
---------- test set results, layer 7
# invariants per class: {0: 3, 1: 3, 2: 4, 3: 6, 4: 3}
support by class: {0: 0.7291666666666666, 1: 0.7142857142857143, 2: 0.6814814814814815, 3: 1.0, 4: 1.0}
precision by class: {0: 1.0, 1: 1.0, 2: 0.989247311827957, 3: 0.4, 4: 1.0}
overall prediction accuracy: 0.667
---------- training set results, layer 8
# invariants per class: {0: 3, 1: 3, 2: 4, 3: 6, 4: 3}
support by class: {0: 0.717391304347826, 1: 0.7021276595744681, 2: 0.7531486146095718, 3: 0.9382716049382716, 4: 0.7878787878787878}
precision by class: {0: 0.9850746268656716, 1: 0.9705882352941176, 2: 0.9835526

It seems that aggregating across spatial dimensions does not improve things. This is somewhat surprising especially in the lower recall results, as one might expect spatial dependencies to narrow the scope of an invariant.

In [8]:
from attribution.methods import InternalInfluence

In [59]:
log_model = Model(model.inputs, model.layers[-2].output)
# log_model.layers[-1].activation = keras.activations.softplus
# log_model = utils.apply_modifications(log_model)

In [60]:
x_tr, x_te = im_tr, im_te

In [61]:
np.random.seed(0)
rand_data = []
cls_size = 256
rand_x = np.random.uniform(size=(cls_size,) + x_tr.shape[1:])
nbiter = 50
for l in range(n_classes):
    model_ch = KerasModel(model)
    sess = K.get_session()
    pgd_params_l2 = {'eps': 5.,
                     'y_target': to_categorical(np.zeros((1,)) + l, n_classes),
                     'eps_iter': 5. / nbiter,
                     'nb_iter': nbiter,
                     'ord': 2,
                     'clip_min': 0.,
                     'clip_max': 1.}
    pgd_params_linf = {'eps': 0.1,
                       'y_target': to_categorical(np.zeros((1,)) + l, n_classes),
                       'eps_iter': 0.1 / nbiter,
                       'nb_iter': nbiter,
                       'ord': np.inf,
                       'clip_min': 0.,
                       'clip_max': 1.}
    pgd = MadryEtAl(model_ch, sess=sess)
    cur_data = []
    cur_data.append(pgd.generate_np(rand_x, **pgd_params_l2))
    cur_data.append(pgd.generate_np(rand_x, **pgd_params_linf))
    rand_data.append(np.concatenate(cur_data, axis=0))
    print('generated class', l)

rand_data = np.concatenate(rand_data)


generated class 0
generated class 1
generated class 2
generated class 3
generated class 4


In [62]:
l7inf = [InternalInfluence(log_model, 7, Q=l, multiply_activation=False, agg_fn=None).compile() for l in range(n_classes)]

In [101]:
l6inf = [InternalInfluence(log_model, 6, Q=l, multiply_activation=False, agg_fn=None).compile() for l in range(n_classes)]

In [103]:
l5inf = [InternalInfluence(log_model, 5, Q=l, multiply_activation=False, agg_fn=None).compile() for l in range(n_classes)]

In [104]:
attr5 = [inf.get_attributions(rand_data, batch_size=1, resolution=100).mean(axis=0) for inf in l5inf]
attr6 = [inf.get_attributions(rand_data, batch_size=1, resolution=100).mean(axis=0) for inf in l6inf]
attr7 = [inf.get_attributions(rand_data, batch_size=1, resolution=100).mean(axis=0) for inf in l7inf]

In [64]:
l7mod = Model(model.input, keras.layers.Flatten()(model.layers[7].output))

In [107]:
x5 = Model(model.input, keras.layers.Flatten()(model.layers[5].output)).predict(x_tr)
x6 = Model(model.input, keras.layers.Flatten()(model.layers[6].output)).predict(x_tr)
x7 = Model(model.input, keras.layers.Flatten()(model.layers[7].output)).predict(x_tr)

In [108]:
w5 = np.concatenate([np.expand_dims(a, 1) for a in attr5], axis=1)
w6 = np.concatenate([np.expand_dims(a, 1) for a in attr6], axis=1)
w7 = np.concatenate([np.expand_dims(a, 1) for a in attr7], axis=1)

In [110]:
w5.shape, x5.shape

((4608, 5), (855, 4608))

In [67]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In [113]:
softmax(sum([np.dot(x, w) for x, w in zip([x5,x6,x7],[w5,w6,w7])]))

array([[6.9167502e-12, 8.8507434e-08, 9.9997497e-01, 2.3316581e-05,
        1.5533250e-06],
       [1.3045338e-08, 1.1155041e-05, 9.8709655e-01, 1.0587788e-02,
        2.3045221e-03],
       [3.8144074e-02, 2.8108570e-03, 8.3436859e-01, 2.1871267e-04,
        1.2445784e-01],
       ...,
       [4.9721816e-10, 7.2655712e-07, 9.9987125e-01, 1.2318733e-04,
        4.7341659e-06],
       [9.9999988e-01, 1.0829607e-07, 3.5423171e-09, 4.7654675e-11,
        7.5672055e-09],
       [1.0000000e+00, 3.6661112e-11, 2.0898318e-11, 4.1094456e-15,
        2.4929795e-08]], dtype=float32)

In [111]:
(softmax(sum([np.dot(x, w) for x, w in zip([x5,x6,x7],[w5,w6,w7])])).argmax(axis=1) == log_model.predict(x_tr).argmax(axis=1)).mean()

0.6865497076023391

In [69]:
model.predict(x_tr)

array([[5.45395380e-13, 1.10029532e-05, 9.99978185e-01, 1.08991335e-05,
        2.16627924e-08],
       [5.48690196e-07, 1.96373183e-03, 3.56685268e-05, 9.97678220e-01,
        3.21844127e-04],
       [9.00300394e-04, 9.99049246e-01, 7.68529958e-07, 4.83345466e-05,
        1.48542347e-06],
       ...,
       [5.25505000e-14, 1.50052230e-07, 9.99992132e-01, 7.71383111e-06,
        1.97880023e-08],
       [1.00000000e+00, 6.47402743e-10, 9.15137420e-25, 7.83871462e-14,
        4.75145910e-13],
       [9.99997735e-01, 1.88389979e-08, 6.19010323e-19, 8.83451891e-15,
        2.30621322e-06]], dtype=float32)

In [70]:
softmax(np.dot(x_l7, w))

array([[3.0218953e-05, 3.8793101e-03, 9.3963283e-01, 4.6191510e-02,
        1.0266158e-02],
       [2.6896663e-04, 1.6604017e-02, 4.5552093e-01, 4.6350417e-01,
        6.4101979e-02],
       [6.2649332e-02, 1.7676464e-01, 4.6528271e-01, 8.2284600e-02,
        2.1301873e-01],
       ...,
       [9.1432215e-05, 4.0450464e-03, 9.4367081e-01, 4.3076344e-02,
        9.1162790e-03],
       [9.6955103e-01, 2.1163097e-02, 2.2962198e-03, 1.6997319e-03,
        5.2899225e-03],
       [9.7016978e-01, 6.9086696e-03, 3.6396983e-03, 4.0939538e-04,
        1.8872451e-02]], dtype=float32)

In [71]:
w.max()

0.16539453

In [72]:
w.min()

-0.20430478

In [73]:
w

array([[-0.01656292, -0.01555725,  0.00356987, -0.02679372, -0.03073645],
       [-0.03200556, -0.05438359, -0.01462941, -0.02353186,  0.01667505],
       [-0.11667184, -0.00488116,  0.0313616 , -0.08707385, -0.09488193],
       ...,
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.02198535,  0.01360314, -0.00521837, -0.01331311, -0.01470261],
       [ 0.01015011, -0.00448576, -0.01429545,  0.00524498, -0.00233526]],
      dtype=float32)

In [74]:
inflinv = InfluenceInvariants(log_model, layer=7, agg_fn=None, multiply_activation=False).compile()
invs = inflinv.get_invariants(rand_data, batch_size=1)

In [75]:
for inv in [inv for inv in invs if inv.Q == 0]:
    print(inv)

(conv2d_4[133] > 0.0 &
 conv2d_4[43] > 0.0 &
 conv2d_4[134] > 0.0 &
 conv2d_4[25] <= -0.5 &
 conv2d_4[11] <= 0.0)
	--> Q = 0
support=0.00193, precision=1.0
(conv2d_4[133] > 0.0 &
 conv2d_4[43] <= 0.0 &
 conv2d_4[35] > 0.0 &
 conv2d_4[1] <= 0.0 &
 conv2d_4[44] <= 0.0)
	--> Q = 0
support=0.0193, precision=1.0
(conv2d_4[133] > 0.0 &
 conv2d_4[43] <= 0.0 &
 conv2d_4[35] <= 0.0 &
 conv2d_4[175] > -0.5)
	--> Q = 0
support=0.973, precision=1.0
(conv2d_4[133] <= 0.0 &
 conv2d_4[139] > 0.0 &
 conv2d_4[216] <= 0.5 &
 conv2d_4[224] > -0.5)
	--> Q = 0
support=0.00578, precision=1.0


In [76]:
inv0 = [inv for inv in invs if inv.Q == 0]

In [86]:
get_invariant_units(inv0[2])

{35, 43, 133, 175}

In [97]:
from attribution.invariant_utils import *
we = np.array([w[u,0] for u in get_invariant_units(inv0[2])])
x_we = x_l7[:,[u for u in get_invariant_units(inv0[2])]]
x_we[:,-1] -= 0.5

In [98]:
we

array([-0.06064251, -0.03093674,  0.00346123,  0.01436928], dtype=float32)

In [100]:
np.where(np.dot(x_we, we) >= 0)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144,
        145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 158,
        159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
        172, 173, 174, 175, 176, 177, 178, 179, 180

In [51]:
x_we

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [37]:
w[:,0].argmax()

6

In [38]:
w[6,0]

0.16476855

In [34]:
log_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
__________